In [4]:
!pip install numpy

   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
    --------------------------------------- 0.3/12.6 MB ? eta -:--:--
    --------------------------------------- 0.3/12.6 MB ? eta -:--:--
    --------------------------------------- 0.3/12.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.6 MB 433.0 kB/s eta 0:00:28
   - -------------------------------------- 0.5/12.6 MB 433.0 kB/s eta 0:00:28
   -- ------------------------------------- 0.8/12.6 MB 518.0 kB/s eta 0:00:23
   -- ------------------------------------- 0.8/12.6 MB 518.0 kB/s eta 0:00:23
   -- ------------------------------------- 0.8/12.6 MB 518.0 kB/s eta 0:00:23
   ---- ----------------------------------- 1.3/12.6 MB 630.1 kB/s eta 0:00:18
   ---- ----------------------------------- 1.3/12.6 MB 630.1 kB/s eta 0:00:18
   ---- ----------------------------------- 1.3/12.6 MB 630.1 kB/s eta 0:00:18
   ---- ----------

In [5]:
import numpy as np

In [6]:
you = np.array([1,0,0,0,0,0,0])
coffee = np.array([0,1,0,0,0,0,0])

In [7]:
W1 = np.array([
    [0.3,0.1,0.1],
    [0.7,0.7,0.7],
    [0.7,0.8,0.3],
    [0.6,0.1,0.1],
    [0.6,0.3,1.0],
    [0.6,0.7,0.6],
    [0.3,0.4,0.2]
])
print(W1)

[[0.3 0.1 0.1]
 [0.7 0.7 0.7]
 [0.7 0.8 0.3]
 [0.6 0.1 0.1]
 [0.6 0.3 1. ]
 [0.6 0.7 0.6]
 [0.3 0.4 0.2]]


In [8]:
h1 = you@W1
print(h1)

[0.3 0.1 0.1]


In [9]:
h2 = coffee@W1
print(h2)

[0.7 0.7 0.7]


In [10]:
h = (h1 + h2)/2
print(h)

[0.5 0.4 0.4]


In [11]:
W2 = np.array([
    [0.2,0.2,0.1,0.6,0.7,0.1,0.5],
    [0.2,0.1,0.5,0.2,0.2,0.2,0.3],
    [0.6,0.1,0.4,0.2,0.8,0.8,0.0]
])

In [12]:
print(W2.T)

[[0.2 0.2 0.6]
 [0.2 0.1 0.1]
 [0.1 0.5 0.4]
 [0.6 0.2 0.2]
 [0.7 0.2 0.8]
 [0.1 0.2 0.8]
 [0.5 0.3 0. ]]


In [13]:
W2.T@h

array([0.42, 0.18, 0.41, 0.46, 0.75, 0.45, 0.37])

In [14]:
np.round(W2.T@h, 0)

array([0., 0., 0., 0., 1., 0., 0.])

# LSTM 실습

In [16]:
! pip install pandas torch

In [17]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [18]:
torch.manual_seed(777)

# ===== 1. CSV에서 데이터 불러오기 =====
df = pd.read_csv("./data/review_data.csv", encoding='cp949')
text = df["text"].tolist()
labels = df["labels"].tolist()

In [19]:
df

,text,labels
0,배우들 연기도 너무 좋았어요.,1
1,스토리가 탄탄하고 연출도 훌륭했어요.,1
2,정말 감동적인 영화였습니다. 눈물이 멈추질 않았어요.,1
3,끝까지 집중해서 봤습니다. 완전 추천해요!,1
4,감성과 메시지가 모두 살아있는 영화였어요.,1
5,오랜만에 이런 좋은 영화를 봐서 기분이 좋아요.,1
6,"음악, 연출, 연기 모두 완벽했어요.",1
7,시간 가는 줄 몰랐어요. 최고의 영화!,1
8,스토리 전개가 매끄럽고 감동적이었어요.,1
9,친구들에게 꼭 추천하고 싶은 영화예요.,1


In [20]:
tokenize = [sentence.split() for sentence in text]
tokenize

[['배우들', '연기도', '너무', '좋았어요.'],
 ['스토리가', '탄탄하고', '연출도', '훌륭했어요.'],
 ['정말', '감동적인', '영화였습니다.', '눈물이', '멈추질', '않았어요.'],
 ['끝까지', '집중해서', '봤습니다.', '완전', '추천해요!'],
 ['감성과', '메시지가', '모두', '살아있는', '영화였어요.'],
 ['오랜만에', '이런', '좋은', '영화를', '봐서', '기분이', '좋아요.'],
 ['음악,', '연출,', '연기', '모두', '완벽했어요.'],
 ['시간', '가는', '줄', '몰랐어요.', '최고의', '영화!'],
 ['스토리', '전개가', '매끄럽고', '감동적이었어요.'],
 ['친구들에게', '꼭', '추천하고', '싶은', '영화예요.'],
 ['지루해서', '중간에', '졸았어요.'],
 ['배우', '연기가', '너무', '어색해서', '몰입이', '안', '됐어요.'],
 ['스토리가', '산만하고', '전개가', '엉성해요.'],
 ['예고편이', '다였네요.', '본편은', '별로예요.'],
 ['너무', '뻔한', '이야기라', '재미가', '없었어요.'],
 ['연출이', '허술하고', '대사도', '부자연스러워요.'],
 ['기대한', '만큼', '실망만', '컸어요.'],
 ['시간', '아깝고', '돈', '아깝네요.'],
 ['한', '번', '보고', '다시는', '보고', '싶지', '않아요.'],
 ['결말도', '어이없고', '전체적으로', '별로였어요.']]

In [21]:
def make_voka(tokenized_sentences):
  voca = set()
  for sentence in tokenized_sentences:
    for word in sentence:
      voca.add(word)
  return voca

voca = make_voka(tokenize)
print(voca)

{'완전', '다였네요.', '영화였어요.', '실망만', '한', '중간에', '정말', '배우들', '감성과', '연출도', '아깝네요.', '음악,', '재미가', '가는', '영화!', '영화를', '연출이', '대사도', '없었어요.', '꼭', '기대한', '연기도', '별로였어요.', '영화였습니다.', '본편은', '연출,', '좋은', '싶지', '연기가', '됐어요.', '줄', '시간', '허술하고', '집중해서', '추천하고', '배우', '눈물이', '탄탄하고', '몰랐어요.', '기분이', '친구들에게', '만큼', '안', '봐서', '모두', '번', '별로예요.', '스토리가', '돈', '감동적인', '끝까지', '엉성해요.', '예고편이', '않았어요.', '결말도', '전개가', '훌륭했어요.', '살아있는', '부자연스러워요.', '어이없고', '최고의', '이런', '추천해요!', '컸어요.', '지루해서', '좋아요.', '뻔한', '아깝고', '싶은', '않아요.', '산만하고', '다시는', '멈추질', '좋았어요.', '감동적이었어요.', '메시지가', '몰입이', '오랜만에', '졸았어요.', '영화예요.', '스토리', '완벽했어요.', '보고', '연기', '어색해서', '봤습니다.', '매끄럽고', '너무', '이야기라', '전체적으로'}


In [22]:
word2idx = {word: idx+1 for idx, word in enumerate(voca)} # padding = 0
print(word2idx)

{'완전': 1, '다였네요.': 2, '영화였어요.': 3, '실망만': 4, '한': 5, '중간에': 6, '정말': 7, '배우들': 8, '감성과': 9, '연출도': 10, '아깝네요.': 11, '음악,': 12, '재미가': 13, '가는': 14, '영화!': 15, '영화를': 16, '연출이': 17, '대사도': 18, '없었어요.': 19, '꼭': 20, '기대한': 21, '연기도': 22, '별로였어요.': 23, '영화였습니다.': 24, '본편은': 25, '연출,': 26, '좋은': 27, '싶지': 28, '연기가': 29, '됐어요.': 30, '줄': 31, '시간': 32, '허술하고': 33, '집중해서': 34, '추천하고': 35, '배우': 36, '눈물이': 37, '탄탄하고': 38, '몰랐어요.': 39, '기분이': 40, '친구들에게': 41, '만큼': 42, '안': 43, '봐서': 44, '모두': 45, '번': 46, '별로예요.': 47, '스토리가': 48, '돈': 49, '감동적인': 50, '끝까지': 51, '엉성해요.': 52, '예고편이': 53, '않았어요.': 54, '결말도': 55, '전개가': 56, '훌륭했어요.': 57, '살아있는': 58, '부자연스러워요.': 59, '어이없고': 60, '최고의': 61, '이런': 62, '추천해요!': 63, '컸어요.': 64, '지루해서': 65, '좋아요.': 66, '뻔한': 67, '아깝고': 68, '싶은': 69, '않아요.': 70, '산만하고': 71, '다시는': 72, '멈추질': 73, '좋았어요.': 74, '감동적이었어요.': 75, '메시지가': 76, '몰입이': 77, '오랜만에': 78, '졸았어요.': 79, '영화예요.': 80, '스토리': 81, '완벽했어요.': 82, '보고': 83, '연기': 84, '어색해서': 85, '봤습니다.': 86, '매끄럽고': 87, '너무': 8

In [23]:
word2idx.get('졸았어요.', 0)

79

In [24]:
word2idx.get('좋았어요.', 0)

74

In [25]:
n_voca = len(word2idx) + 1
print(n_voca)

91


In [26]:
# ======= 3. Dataset 클래스 생성 =======
class TextDataset(Dataset):
  def __init__(self, text, labels, word2idx):
    self.data = [sc.split() for sc in text]
    self.labels = labels
    self.word2idx = word2idx
    self.max_len = max(len(sc) for sc in self.data)

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    sentence = self.data[idx]
    indices = [word2idx.get(word,0) for word in sentence]
    padding = indices + [0] * (self.max_len - len(indices))
    return torch.tensor(padding), torch.tensor(self.labels[idx])


In [27]:
dataset = TextDataset(text, labels, word2idx)
loader = DataLoader(dataset, batch_size=2, shuffle=True)

In [28]:
# ======= 4. LSTM 모델 =======
class LSTMClassifier(nn.Module):
  def __init__(self, n_voca, embed_dim, hidden_dim, output_dim):
    super().__init__()
    self.embedding = nn.Embedding(n_voca, embed_dim, padding_idx=0)
    self.lstm = nn.LSTM(
        input_size = embed_dim,
        hidden_size = hidden_dim,
        num_layers = 2,
        batch_first = True
    )
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, x):
    embedded = self.embedding(x)
    _, (hidden, _) = self.lstm(embedded)
    return self.fc(hidden[-1])

In [29]:
model = LSTMClassifier(n_voca, embed_dim=16, hidden_dim=32, output_dim=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [30]:
# ======= 5. 학습 =======
for epoch in range(10):
  for x_batch, y_batch in loader:
    optimizer.zero_grad()
    y_hat = model(x_batch)
    loss = criterion(y_hat, y_batch)
    loss.backward()
    optimizer.step()
  print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 1, Loss: 0.5875
Epoch 2, Loss: 0.6931
Epoch 3, Loss: 0.7417
Epoch 4, Loss: 0.9228
Epoch 5, Loss: 0.0488
Epoch 6, Loss: 0.0012
Epoch 7, Loss: 0.0004
Epoch 8, Loss: 0.0004
Epoch 9, Loss: 0.0003
Epoch 10, Loss: 0.0003


In [31]:
# ======= 6. 예측 =======
def predict(sentence):
  model.eval()
  with torch.no_grad():
    words = sentence.split()
    indices = [word2idx.get(word, 0) for word in words]
    padding = indices + [0] * (dataset.max_len - len(indices))
    input_tensor = torch.tensor([padding])
    output = model(input_tensor)
    predicted = torch.argmax(output, dim=1).item()
    return "긍정" if predicted == 1 else "부정"

In [32]:
# ======= 7. 예측 테스트 =======
test_sentences = [
    "정말 감동적인 영화였어",
    "지루하고 재미없다",
    "다시 보고 싶어",
    "정말 별로 재미없어. 졸았어"
]

for sentence in test_sentences:
  print(f"문장: {sentence} -> 예측: {predict(sentence)}")

문장: 정말 감동적인 영화였어 -> 예측: 긍정
문장: 지루하고 재미없다 -> 예측: 긍정
문장: 다시 보고 싶어 -> 예측: 부정
문장: 정말 별로 재미없어. 졸았어 -> 예측: 긍정
